In [1]:
import pycolmap


In [2]:
from pathlib import Path

In [3]:
!nvidia-smi

Fri Mar 15 09:23:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX150           Off | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8              N/A /   9W |      1MiB /  2048MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Prepare Paths

In [4]:
data_path = Path("../data/images")
colmap_path = Path("../data/colmap")

data_path.mkdir(parents=True, exist_ok=True)

# colmap directory should be empty
# colmap_path.mkdir(parents=True, exist_ok=True)
# import shutil
# shutil.rmtree(str(colmap_path))
# colmap_path.mkdir()

In [5]:
output_path=  colmap_path
image_dir =  data_path

mvs_path = output_path / "mvs"
database_path = output_path / "database.db"


# match and extract features

In [ ]:
pycolmap.extract_features(database_path, image_dir)
pycolmap.match_exhaustive(database_path)

In [ ]:
maps = pycolmap.incremental_mapping(database_path, image_dir, output_path)
maps[0].write(output_path)


In [ ]:
# dense reconstruction
pycolmap.undistort_images(mvs_path, output_path, image_dir)
pycolmap.patch_match_stereo(mvs_path)  # requires compilation with CUDA
pycolmap.stereo_fusion(mvs_path / "dense.ply", mvs_path)

In [ ]:
import pycolmap

# Load the COLMAP database
db = pycolmap.Database(str(database_path))

In [ ]:
tvg = db.read_two_view_geometry(1,2)

In [ ]:
img = db.read_all_images()[0]

In [ ]:
img = db.read_image(3)
img.todict()['points2D']